## 2.2 - Chain of Thought Prompting

**Chain of Thought** (CoT) is a concept in natural language processing (NLP) that enables 
language models to reason through tasks in a structured, step-by-step manner, similar to human 
thinking. Rather than generating isolated responses, a model using Chain of Thought builds on 
prior information, creating responses that follow a logical flow and ensuring a coherent, 
context-aware dialogue. This approach is particularly valuable in complex or multi-step queries,
where understanding and maintaining context over multiple interactions is essential.

## Why Chain of Thought Matters

In traditional LLM models, responses can sometimes feel disjointed or context-free, especially with complex questions. Chain of Thought addresses these challenges by adding a reasoning layer, allowing the model to generate more accurate and relevant responses. This logical progression offers key benefits:

1. **Coherence**: Responses follow a natural flow, making it easier for users to follow and engage with the conversation.
2. **Context Retention**: By building on previous information, the model maintains context across multiple steps, crucial for handling complex questions.
3. **Increased Accuracy**: Logical progression reduces the likelihood of irrelevant or incorrect information, enhancing the reliability of responses.


## Preparation

In [1]:
# !pip install -q langchain openai
# !pip install langchain-openai
# !pip install "langchain-core==0.3.27"
# !pip install langchain_community
!pip install -q langchain-openai termcolor langchain_community duckduckgo_search wikipedia openapi-python-client langgraph langchain_experimental openai --quiet

In [2]:
# Imports
import json
import os
from os import listdir
from os.path import isfile, join
from langchain.chains import LLMChain
#from langchain_community.llms import VLLMOpenAI
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

In [3]:
# INFERENCE_SERVER_URL = os.getenv('API_URL_GRANITE')
# MODEL_NAME = "granite-3-8b-instruct"
# API_KEY= os.getenv('API_KEY')

INFERENCE_SERVER_URL = "http://localhost:8000"
MODEL_NAME = "ibm-granite/granite-3.0-8b-instruct"
API_KEY= "alanliuxiang"

# INFERENCE_SERVER_URL = "https://api.feidaapi.com"
# MODEL_NAME = "gpt-4"
# API_KEY= "sk-TsoMJYTiW1Ya1AatboGdUGXEdCOAWeCeIbvUCkNMonrCHoQ7"

### Create the LLM instance

In [4]:
# LLM definition
llm = ChatOpenAI(
    openai_api_key=API_KEY,
    openai_api_base= f"{INFERENCE_SERVER_URL}/v1",
    model_name=MODEL_NAME,
    top_p=0.92,
    temperature=0.01,
    max_tokens=512,
    presence_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

### Create the Prompt

In [5]:
template = """\
<|start_of_role|>system<|end_of_role|>
You are a helpful, respectful, and honest assistant. Always be as helpful as possible, while being safe. 
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. 
Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something incorrect. 
If you don't know the answer to a question, please don't share false information.
<|end_of_text|>

<|start_of_role|>user<|end_of_role|>
Human: {input}
<|end_of_text|>

<|start_of_role|>assistant<|end_of_role|>
"""

PROMPT = PromptTemplate(input_variables=["input"], template=template)

In [6]:
conversation = LLMChain(llm=llm,
                            prompt=PROMPT,
                            verbose=False,
                            )

/tmp/ipykernel_2384/595551351.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  conversation = LLMChain(llm=llm,


### Let's ask a simple question **WITHOUT** Chain of Thought in action

In [7]:
first_input = "I want to design a finantial report around the report from the larger FSIs. What I should focus?"
conversation.predict(input=first_input);

When designing a financial report, especially one that is based on reports from large Financial Services Institutions (FSIs), there are several key areas you should focus on:

1. **Executive Summary**: This section provides an overview of the entire report. It should be concise and highlight the most important findings.

2. **Financial Highlights**: This section includes key financial data such as revenue, net income, earnings per share, dividends, etc. It's also useful to include year-over-year comparisons and industry benchmarks.

3. **Balance Sheet**: This provides information about the company's assets, liabilities, and shareholders' equity.

4. **Income Statement**: This shows how much revenue the company earned over a specific period of time, as well as the costs and expenses associated with earning that revenue.

5. **Cash Flow Statement**: This shows how changes in balance sheet accounts and income affect cash and cash equivalents, and breaks the analysis down to operating, inv

### LLM with Chain of Thought in the Prompt

In [8]:
template="""<s>[INST]<<SYS>>
You are a helpful, respectful, and honest assistant. Always be as helpful as possible while being safe.
You will be asked a question, to which you must give an answer.
Your answer should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.
When answering, think step-by-step and explain your reasoning before providing the final answer.
Explain it with bullet points if it's needed but using reasoning and step-by-step.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something incorrect.
If you don't know the answer to a question, answer "I don't know".
<</SYS>>

### QUESTION:
{input}

### ANSWER:
[/INST]
"""
PROMPT_COT = PromptTemplate(input_variables=["input"], template=template)

### Chain of Thought Prompt

Let's ask the same question with Chain of Thought

* When answering, think step-by-step and explain your reasoning before providing the final answer.
* Explain it with bullet points if it's needed but using reasoning and step-by-step.

In [9]:
conversation_cot = LLMChain(llm=llm,
                            prompt=PROMPT_COT,
                            verbose=False,
                            )

In [10]:
first_input = "I want to design a finantial report around the report from the larger FSIs. What I should focus?"
conversation_cot.predict(input=first_input);

When designing a financial report around the reports from larger Financial Services Institutions (FSIs), you should focus on several key areas:

1. **Financial Performance**: This includes revenue, profit, and loss statements. It's important to understand how these institutions are performing financially, as it can provide insights into their business strategies and market conditions.

2. **Balance Sheet Analysis**: This involves looking at assets, liabilities, and shareholders' equity. It gives a snapshot of what the institution owns and owes, as well as the amount invested by shareholders.

3. **Cash Flow Statement**: This shows how changes in balance sheet accounts and income affect cash and cash equivalents, and breaks the analysis down to operating, investing, and financing activities.

4. **Risk Management**: FSIs face various types of risks including credit risk, market risk, operational risk, etc. Understanding how they manage these risks can be crucial.

5. **Regulatory Compli

## Enhancing LLMs Performance with Chain of Thought

By simulating a thought process, Chain of Thought enhances NLP models by:

- **Improving Context Understanding**: The model remembers previous interactions, generating responses that are relevant and contextually aware.
- **Boosting User Experience**: Users benefit from a more natural and satisfying interaction, as the model “thinks” through its responses.
- **Increasing Flexibility**: CoT enables models to handle multi-part questions, making them versatile for applications like virtual assistants, customer service bots, and educational tools.

Incorporating Chain of Thought into LLMs frameworks, such as LangChain, enables language models to tackle complex tasks, making them more effective for real-world applications where logical reasoning and clarity are paramount.